In [9]:
# 필요한 패키지 설치
!apt update && apt install chromium-chromedriver
!pip install selenium
!pip install pandas

# 라이브러리 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re
from google.colab import drive
import os

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,721 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,994 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,953 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

In [10]:
def setup_driver():
    """구글 코랩용 Chrome 드라이버 설정"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [11]:
#잡코리아 로그인

def login_to_jobkorea(driver, user_id, password):
    """수정된 잡코리아 로그인 함수 - 정확한 URL과 요소 사용"""
    try:
        print("잡코리아 로그인 페이지로 이동...")

        # 제공해주신 정확한 로그인 URL 사용
        login_url = "https://www.jobkorea.co.kr/Login/Login_Tot.asp?rDBName=GG&re_url=%2f%3futm_source%3dgoogle%26utm_medium%3dcpc%26utm_campaign%3dpc_%25EB%25B8%258C%25EB%259E%259C%25EB%2593%259C_tcpa%26utm_content%3dpc_%25EB%25B8%258C%25EB%259E%259C%25EB%2593%259C_tcpa_%25EC%259E%25A1%25EC%25BD%2594%25EB%25A6%25AC%25EC%2595%25841%25EC%259C%2584%26utm_term%3d%25EC%259E%25A1%2b%25EC%25BD%2594%25EB%25A6%25AC%25EC%2595%2584%26gad_source%3d1%26gad_campaignid%3d20025807087%26gbraid%3d0AAAAAD21Aaf0ARAL_GDIQvUnRLyWmkSW5%26gclid%3dCjwKCAjw6NrBBhB6EiwAvnT_rnlGOoB2Y6UjvKVTJK4tP1wdhoeflXz9B3pzLPo5Xt4mgWH3S9JSjBoCH14QAvD_BwE"

        driver.get(login_url)
        time.sleep(5)

        print(f"페이지 제목: {driver.title}")
        print(f"현재 URL: {driver.current_url}")

        # 개인회원 탭 클릭
        try:
            personal_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '개인회원')]"))
            )
            driver.execute_script("arguments[0].click();", personal_tab)
            time.sleep(2)
            print("개인회원 탭 클릭 완료")
        except:
            print("개인회원 탭이 이미 선택되어 있거나 찾을 수 없음")

        # ID 입력 필드 찾기 - 플레이스홀더 "NO.1 잡코리아·알바몬 통합 ID"
        id_input = None
        id_selectors = [
            "input[placeholder*='통합 ID']",
            "input[placeholder*='잡코리아']",
            "input[placeholder*='알바몬']"
        ]

        for selector in id_selectors:
            try:
                id_input = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                )
                print(f"ID 입력 필드 발견: {selector}")
                break
            except:
                continue

        if not id_input:
            # 백업 방법: 첫 번째 텍스트 입력 필드
            try:
                id_input = driver.find_element(By.CSS_SELECTOR, "input[type='text']")
                print("ID 입력 필드 발견 (백업 방법)")
            except:
                print("❌ ID 입력 필드를 찾을 수 없습니다.")
                return False

        # 비밀번호 입력 필드 찾기
        try:
            pw_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='password']"))
            )
            print("비밀번호 입력 필드 발견")
        except:
            print("❌ 비밀번호 입력 필드를 찾을 수 없습니다.")
            return False

        # 로그인 정보 입력
        print("로그인 정보 입력 중...")
        id_input.clear()
        id_input.send_keys(user_id)
        time.sleep(1)

        pw_input.clear()
        pw_input.send_keys(password)
        time.sleep(1)

        # 로그인 버튼 찾기 - 검색 결과에서 "로그인" 텍스트를 가진 버튼
        login_btn = None
        login_btn_selectors = [
            "//button[text()='로그인']",
            "//input[@value='로그인']",
            "//button[contains(text(), '로그인')]",
            "//a[contains(text(), '로그인')]"
        ]

        for selector in login_btn_selectors:
            try:
                login_btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, selector))
                )
                print(f"로그인 버튼 발견: {selector}")
                break
            except:
                continue

        if login_btn:
            print("로그인 버튼 클릭...")
            driver.execute_script("arguments[0].click();", login_btn)
        else:
            print("로그인 버튼을 찾을 수 없어 Enter 키로 시도")
            from selenium.webdriver.common.keys import Keys
            pw_input.send_keys(Keys.RETURN)

        # 로그인 처리 대기
        time.sleep(10)

        # 로그인 성공 확인
        current_url = driver.current_url
        page_title = driver.title

        print(f"로그인 후 URL: {current_url}")
        print(f"로그인 후 제목: {page_title}")

        # 로그인 성공 여부 판단 (Login이 URL에 없고, 로그인이 제목에 없으면 성공)
        success_indicators = [
            "Login" not in current_url,
            "로그인" not in page_title,
            "jobkorea.co.kr" in current_url and "Login" not in current_url
        ]

        if any(success_indicators):
            print("✅ 로그인 성공!")
            return True
        else:
            print("❌ 로그인 실패")
            # 에러 메시지 확인
            try:
                error_selectors = [".error", ".alert", "[class*='error']", "[class*='message']"]
                for selector in error_selectors:
                    error_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    for elem in error_elements:
                        if elem.text.strip():
                            print(f"에러 메시지: {elem.text}")
            except:
                pass
            return False

    except Exception as e:
        print(f"로그인 오류: {e}")
        return False


In [12]:
# 회사명 추출
def extract_company_name(driver):
    company_name = ""

    try:
        # 방법 1: 기업 홈 이동 링크에서 추출
        try:
            # 기업 홈 이동 링크 찾기
            company_link_selectors = [
                'a[title="기업 홈 이동"]',
                'a[href*="/Recruit/Co_Read/C/"]',
                'a[target="_blank"][title="기업 홈 이동"]'
            ]

            for selector in company_link_selectors:
                try:
                    company_link = driver.find_element(By.CSS_SELECTOR, selector)
                    company_name = company_link.text.strip()
                    if company_name and len(company_name) > 1:
                        print(f"기업 홈 링크에서 회사명 추출: {company_name}")
                        return company_name
                except:
                    continue
        except Exception as e:
            print(f"기업 홈 링크에서 회사명 추출 오류: {e}")

        print("❌ 회사명을 찾을 수 없습니다.")
        return ""

    except Exception as e:
        print(f"회사명 추출 전체 오류: {e}")
        return ""

In [13]:
# 직무 관련 텍스트 추출
def extract_position_text(driver):

    position_text = ""

    try:
        # em 태그에서 텍스트 추출
        em_elements = driver.find_elements(By.TAG_NAME, "em")
        for elem in em_elements:
            try:
                text = elem.text.strip()
                if text and len(text) > 3:
                    position_text = text
                    print(f"em 태그에서 텍스트 추출: {text}")
                    break
            except:
                continue

        # em 태그에서 못 찾으면 제목에서 추출
        if not position_text:
            try:
                position_text = driver.title or ""
                print(f"제목에서 텍스트 추출: {position_text}")
            except:
                position_text = ""

    except Exception as e:
        print(f"텍스트 추출 오류: {e}")
        position_text = ""

    return position_text


In [14]:
# 직무 텍스트 처리
def extract_position_info(text):

    year_period = ""
    status = ""
    job = ""

    # 연도/기간 추출
    year_period_match = re.search(r"\d{4}년?\s*[상하]반기", text)
    if year_period_match:
        year_period = year_period_match.group(0)

    # 상태 추출 (인턴, 신입만)
    status_keywords = ["인턴", "신입"]
    for keyword in status_keywords:
        if keyword in text:
            status = keyword
            break

    # 직업 추출 - 검색 결과의 방식 적용
    words = text.split()
    for word in words:
        # 상태 키워드 제외
        if word in status_keywords:
            continue
        # 연도/기간 관련 단어 제외
        if any(time_word in word for time_word in ["년", "반기", "202"]):
            continue
        # 길이가 2글자 이상이고 한글, 영문, 숫자, 특수문자 포함
        if len(word) >= 2 and re.match(r'^[\w가-힣·&]+$', word):
            job = word
            break

    return year_period, status, job

In [15]:
#질문/ 답변 추출 및 불필요한 데이터 제거
def extract_clean_qa_pairs(driver):
    questions = []
    answers = []

    try:
        # 자소서 본문 영역만 선택
        main_content = None
        content_selectors = [
            ".qnaLists",              # 메인 Q&A 리스트
            ".passAssayContent",      # 자소서 내용
            ".essay_content",         # 에세이 내용
            "#passAssayQnaLists",     # Q&A 리스트 ID
            ".passAssayView .qna"     # 자소서 뷰의 Q&A
        ]

        for selector in content_selectors:
            try:
                main_content = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"메인 콘텐츠 영역 발견: {selector}")
                break
            except:
                continue

        if not main_content:
            print("메인 콘텐츠 영역을 찾을 수 없어 전체 페이지에서 추출 시도")
            main_content = driver.find_element(By.TAG_NAME, "body")

        # dt(질문), dd(답변) 태그 쌍으로 추출
        question_elements = main_content.find_elements(By.TAG_NAME, 'dt')
        answer_elements = main_content.find_elements(By.TAG_NAME, 'dd')

        print(f"질문 요소 수: {len(question_elements)}, 답변 요소 수: {len(answer_elements)}")

        for i in range(min(len(question_elements), len(answer_elements))):
            try:
                # 질문 처리
                question_elem = question_elements[i]

                # 펼치기 버튼이 있으면 클릭
                try:
                    question_btn = question_elem.find_element(By.TAG_NAME, 'button')
                    driver.execute_script("arguments[0].click();", question_btn)
                    time.sleep(1)
                except:
                    pass

                # 질문 텍스트 추출
                question_text = extract_text_from_element(question_elem)

                # 답변 처리
                answer_elem = answer_elements[i]

                # 답변 펼치기 버튼이 있으면 클릭
                try:
                    answer_btn = answer_elem.find_element(By.TAG_NAME, 'button')
                    driver.execute_script("arguments[0].click();", answer_btn)
                    time.sleep(1)
                except:
                    pass

                # 답변 텍스트 추출
                answer_text = extract_text_from_element(answer_elem)

                # 유효한 질문/답변인지 확인 (엄격한 필터링)
                if is_valid_essay_qa(question_text, answer_text):
                    questions.append(question_text)
                    answers.append(answer_text)
                    print(f"유효한 Q{len(questions)}: {question_text[:30]}...")
                else:
                    print(f"무효한 데이터 제외: Q='{question_text[:30]}', A='{answer_text[:30]}'")

            except Exception as e:
                print(f"Q&A {i+1} 처리 오류: {e}")
                continue

        return questions, answers

    except Exception as e:
        print(f"Q&A 추출 오류: {e}")
        return [], []

# 텍스트 추출
def extract_text_from_element(element):

    try:
        # .tx 클래스가 있으면 우선 사용
        tx_elem = element.find_element(By.CLASS_NAME, 'tx')
        return tx_elem.text.strip()
    except:
        # 전체 텍스트 사용
        return element.text.strip()

# 유효한 자소서인지 검증
def is_valid_essay_qa(question, answer):

    # 1차: 불필요한 키워드 완전 차단
    invalid_keywords = [
        '공지사항', 'FAX', 'Email', 'email', '전화번호', '고객센터',
        '이용약관', '개인정보', '잡코리아', '알바몬', 'jobkorea',
        '02-565-9351', '평일 09:00', '토요일 09:00', '19:00', '15:00',
        '설문이벤트', '당첨자', '발표', '채용담당자', '채용 트렌드',
        '키워드', '사칭 연락', '주의하세요', 'YYYYMMDD', '25.05',
        '(평일', '토요일)', 'x알바몬', '무료'
    ]

    question_lower = question.lower()
    answer_lower = answer.lower()

    for keyword in invalid_keywords:
        if keyword.lower() in question_lower or keyword.lower() in answer_lower:
            return False

    # 2차: 최소 길이 확인 (너무 짧은 것 제외)
    if len(question.strip()) < 15 or len(answer.strip()) < 30:
        return False

    # 3차: 실제 자소서 질문 패턴 확인
    valid_question_indicators = [
        'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10',
        '질문', '지원동기', '입사후', '입사 후', '성장과정', '성격의 장점',
        '성격의 단점', '장점', '단점', '경험', '역량', '포부', '계기',
        '학창시절', '대학시절', '프로젝트', '성공경험', '실패경험',
        '어려움', '극복', '리더십', '팀워크', '갈등', '해결', '도전',
        '목표', '비전', '직무', '회사', '기업', '조직', '업무',
        '능력', '전문성', '기술', '지식', '문제해결', '창의', '혁신',
        '소통', '협업', '책임감', '열정', '성취', '발전', '개선',
        '변화', '시도', '노력', '준비', '계획', '전략', '방법'
    ]

    # 질문에 유효한 패턴이 포함되어 있는지 확인
    has_valid_pattern = any(indicator in question for indicator in valid_question_indicators)

    # 4차: 답변 품질 확인 (실제 자소서 답변 같은지)
    answer_quality_indicators = [
        '경험', '프로젝트', '활동', '업무', '학습', '성장', '발전',
        '노력', '도전', '성과', '결과', '성공', '달성', '완성',
        '개선', '해결', '극복', '협업', '소통', '리더십', '책임',
        '열정', '목표', '계획', '전략', '방법', '과정', '단계'
    ]

    has_quality_answer = any(indicator in answer for indicator in answer_quality_indicators)

    # 5차: 숫자나 날짜만 있는 것 제외
    if answer.replace(' ', '').replace('-', '').replace(':', '').isdigit():
        return False

    return has_valid_pattern and has_quality_answer


In [16]:
# Essay Data 형태 생성
def extract_essay_content(driver, essay_url):
    try:
        driver.get(essay_url)
        time.sleep(5)

        essay_data = {
            'url': essay_url,
            'company': '',
            'year_period': '',
            'status': '',
            'job': '',
            'questions': [],
            'answers': [],
            'essay_id': ''
        }

        # URL에서 essay_id 추출
        match = re.search(r'/View/(\d+)', essay_url)
        if match:
            essay_data['essay_id'] = match.group(1)

        # 회사명 추출
        essay_data['company'] = extract_company_name(driver)

        # 직무 관련 텍스트 추출
        position_text = extract_position_text(driver)

        # 직무 정보 파싱 (안전한 언패킹)
        try:
            year_period, status, job = extract_position_info(position_text)
            essay_data['year_period'] = year_period
            essay_data['status'] = status
            essay_data['job'] = job
            print(f"추출된 정보 - 기간: '{year_period}', 상태: '{status}', 직업: '{job}'")
        except ValueError as e:
            print(f"언패킹 오류: {e}")
            essay_data['year_period'] = ""
            essay_data['status'] = ""
            essay_data['job'] = ""

        # 실제 자소서 Q&A만 추출
        questions, answers = extract_clean_qa_pairs(driver)
        essay_data['questions'] = questions
        essay_data['answers'] = answers

        return essay_data if questions else None

    except Exception as e:
        print(f"자소서 내용 추출 오류: {e}")
        # 기본값으로 essay_data 반환
        return {
            'url': essay_url,
            'company': '',
            'year_period': '',
            'status': '',
            'job': '',
            'questions': [],
            'answers': [],
            'essay_id': ''
        }

In [17]:
# 자소서 스크래핑
def scrape_jobkorea_with_login(user_id, password, start_page, end_page):
    """시작페이지와 끝페이지로 잡코리아 스크래핑"""
    driver = setup_driver()
    all_essays = []

    try:
        # 로그인
        print("=== 로그인 시도 ===")
        if not login_to_jobkorea(driver, user_id, password):
            print("로그인 실패. 프로그램을 종료합니다.")
            return []

        print("\n=== 합격자소서 페이지로 이동 ===")
        base_url = "https://www.jobkorea.co.kr/starter/passassay?schTxt=&Page="

        for page in range(start_page, end_page + 1):
            print(f"\n페이지 {page}/{end_page} 처리 중...")

            page_url = base_url + str(page)
            driver.get(page_url)
            time.sleep(5)

            # 자소서 링크 찾기
            essay_links = get_essay_links_from_page(driver, page_url)

            if not essay_links:
                print("자소서 링크를 찾을 수 없습니다.")
                if "로그인" in driver.page_source:
                    print("로그인이 필요한 페이지입니다.")
                    break
                continue

            # 각 자소서 처리
            for i, essay_url in enumerate(essay_links):
                print(f"\n  {i+1}/{min(len(essay_links), 3)} 자소서 처리...")
                print(f"  URL: {essay_url}")

                # 개선된 추출 함수 사용
                essay_data = extract_essay_content(driver, essay_url)

                if essay_data and essay_data['questions']:
                    all_essays.append(essay_data)
                    print(f"    ✅ 성공!")
                    print(f"       회사: '{essay_data['company']}'")
                    print(f"       연도: '{essay_data['year_period']}'")
                    print(f"       상태: '{essay_data['status']}'")
                    print(f"       직업: '{essay_data['job']}'")
                    print(f"       문항수: {len(essay_data['questions'])}")
                else:
                    print(f"    ❌ 실패 또는 유효한 데이터 없음")

                time.sleep(3)

    except Exception as e:
        print(f"스크래핑 오류: {e}")

    finally:
        driver.quit()

    return all_essays

# 페이지에서 자소서 링크 추출 함수 (기존과 동일)
def get_essay_links_from_page(driver, page_url):
    try:
        essay_links = []

        # 여러 선택자로 자소서 링크 찾기
        selectors = [
            "a[href*='/starter/PassAssay/View/']",
            "a[href*='PassAssay/View']",
            "a[href*='passassay/View']",
            "a[href*='passassay/view']"
        ]

        for selector in selectors:
            try:
                links = driver.find_elements(By.CSS_SELECTOR, selector)
                for link in links:
                    href = link.get_attribute('href')
                    if href and ('PassAssay/View' in href or 'passassay/View' in href or 'passassay/view' in href):
                        essay_links.append(href)

                if essay_links:
                    break
            except:
                continue

        # 중복 제거
        essay_links = list(set(essay_links))
        print(f"페이지에서 {len(essay_links)}개 자소서 링크 발견")

        return essay_links

    except Exception as e:
        print(f"링크 추출 오류: {e}")
        return []

In [ ]:
# 실행 코드 - 단일 CSV 저장 버전
if __name__ == "__main__":
    import datetime
    import os

    # 구글 드라이브 마운트
    from google.colab import drive
    drive.mount('/content/drive')

    # 저장 경로 설정
    save_directory = '/content/drive/MyDrive/Colab Notebooks/NLP/Project/'
    os.makedirs(save_directory, exist_ok=True)

    # 로그인 정보
    user_id = "kastor0323"
    password = "20gkrqjs!"

    # 스크래핑 페이지 범위 설정
    start_page = 151
    end_page = 250  # 원하는 끝 페이지로 설정

    # 스크래핑 실행
    print(f"\n=== 잡코리아 합격자소서 스크래핑 ({start_page}~{end_page}페이지) ===")
    start_time = datetime.datetime.now()

    try:
        essays = scrape_jobkorea_with_login(user_id, password, start_page=start_page, end_page=end_page)

        if essays:
            print(f"\n✅ 총 {len(essays)}개의 자소서를 수집했습니다!")

            # 데이터 변환
            df_data = []
            successful_essays = 0

            for essay in essays:
                if essay and essay['questions'] and essay['answers']:
                    successful_essays += 1
                    for i, (question, answer) in enumerate(zip(essay['questions'], essay['answers'])):
                        df_data.append({
                            'essay_id': essay['essay_id'],
                            'company': essay['company'],
                            'year_period' : essay['year_period'],
                            'status' : essay['status'],
                            'job' : essay['job'],
                            'question_num': i + 1,
                            'question': question,
                            'answer': answer,
                            'url': essay['url']
                        })

            if df_data:
                df = pd.DataFrame(df_data)

                # 데이터 품질 검증
                print(f"\n=== 데이터 품질 상세 분석 ===")
                print(f"총 행 수: {len(df)}")
                print(f"성공적으로 처리된 자소서: {successful_essays}/{len(essays)}")

                # 회사명 분석
                companies_filled = df[df['company'].str.len() > 0]
                print(f"회사명 채워진 행: {len(companies_filled)}/{len(df)} ({len(companies_filled)/len(df)*100:.1f}%)")

                if len(companies_filled) > 0:
                    unique_companies = df['company'].value_counts()
                    print(f"유니크 회사 수: {len(unique_companies)}")
                    print("상위 회사들:")
                    for company, count in unique_companies.head(5).items():
                        if company:
                            print(f"  - {company}: {count}개 문항")

                # 질문/답변 길이 분석
                avg_question_len = df['question'].str.len().mean()
                avg_answer_len = df['answer'].str.len().mean()
                print(f"평균 질문 길이: {avg_question_len:.0f}자")
                print(f"평균 답변 길이: {avg_answer_len:.0f}자")

                # 데이터 정제 검증
                print(f"\n=== 데이터 정제 검증 ===")
                bad_keywords = ['공지사항', 'FAX', 'Email', '02-565-9351', '평일 09:00',
                              '잡코리아', '알바몬', '토요일', '설문이벤트', '당첨자']

                total_bad_data = 0
                for keyword in bad_keywords:
                    bad_q = df[df['question'].str.contains(keyword, na=False, case=False)]
                    bad_a = df[df['answer'].str.contains(keyword, na=False, case=False)]

                    if len(bad_q) > 0 or len(bad_a) > 0:
                        print(f"⚠️ '{keyword}' 포함된 데이터: 질문 {len(bad_q)}개, 답변 {len(bad_a)}개")
                        total_bad_data += len(bad_q) + len(bad_a)

                if total_bad_data == 0:
                    print("✅ 불필요한 데이터 완전 제거됨!")
                else:
                    print(f"⚠️ 총 {total_bad_data}개의 불필요한 데이터 발견")

                # CSV 파일 저장 (2개 파일)
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

                # 1. 타임스탬프 포함된 파일
                csv_filename_detailed = f'jobkorea_essays_p{start_page}-{end_page}_{timestamp}.csv'
                csv_path_detailed = os.path.join(save_directory, csv_filename_detailed)
                df.to_csv(csv_path_detailed, index=False, encoding='utf-8-sig')

                # 2. 고정된 파일명
                csv_filename_simple = 'jobkorea_essays.csv'
                csv_path_simple = os.path.join(save_directory, csv_filename_simple)
                df.to_csv(csv_path_simple, index=False, encoding='utf-8-sig')

                print(f"\n💾 데이터 저장 완료:")
                print(f"   📄 상세 파일: {csv_filename_detailed}")
                print(f"   📄 기본 파일: {csv_filename_simple}")

                # 완료 시간 계산
                end_time = datetime.datetime.now()
                duration = end_time - start_time

                print(f"\n🎉 스크래핑 완료!")
                print(f"⏱️ 소요 시간: {duration}")
                print(f"📊 수집 페이지 범위: {start_page}~{end_page}")
                print(f"📝 총 자소서 수: {len(essays)}")
                print(f"📋 총 Q&A 수: {len(df_data)}")
                print(f"💾 저장 파일: {csv_filename_detailed}, {csv_filename_simple}")

                # 수집된 데이터 샘플
                print(f"\n=== 수집된 데이터 샘플 (상위 3개) ===")
                sample_count = 0
                for idx, row in df.iterrows():
                    if sample_count >= 3:
                        break
                    if row['company']:
                        sample_count += 1
                        print(f"\n[샘플 {sample_count}]")
                        print(f"회사: {row['company']}")
                        print(f"연도: {row['year_period']}")
                        print(f"상태: {row['status']}")
                        print(f"직업: {row['job']}")
                        print(f"질문: {row['question'][:80]}...")
                        print(f"답변: {row['answer'][:80]}...")

        else:
            print("❌ 스크래핑된 자소서가 없습니다.")
            print("로그인 정보나 페이지 구조를 확인해주세요.")

    except Exception as e:
        print(f"❌ 실행 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

    finally:
        print(f"\n=== 프로그램 종료 ===")
        end_time = datetime.datetime.now() if 'end_time' not in locals() else end_time
        total_duration = end_time - start_time
        print(f"총 실행 시간: {total_duration}")


Mounted at /content/drive

=== 잡코리아 합격자소서 스크래핑 (151~250페이지) ===
=== 로그인 시도 ===
잡코리아 로그인 페이지로 이동...
페이지 제목: 잡코리아·알바몬 통합 로그인 ㅣ 잡코리아 - 좋은 일이 생긴다
현재 URL: https://www.jobkorea.co.kr/Login/Login_Tot.asp?rDBName=GG&re_url=%2f%3futm_source%3dgoogle%26utm_medium%3dcpc%26utm_campaign%3dpc_%25EB%25B8%258C%25EB%259E%259C%25EB%2593%259C_tcpa%26utm_content%3dpc_%25EB%25B8%258C%25EB%259E%259C%25EB%2593%259C_tcpa_%25EC%259E%25A1%25EC%25BD%2594%25EB%25A6%25AC%25EC%2595%25841%25EC%259C%2584%26utm_term%3d%25EC%259E%25A1%2b%25EC%25BD%2594%25EB%25A6%25AC%25EC%2595%2584%26gad_source%3d1%26gad_campaignid%3d20025807087%26gbraid%3d0AAAAAD21Aaf0ARAL_GDIQvUnRLyWmkSW5%26gclid%3dCjwKCAjw6NrBBhB6EiwAvnT_rnlGOoB2Y6UjvKVTJK4tP1wdhoeflXz9B3pzLPo5Xt4mgWH3S9JSjBoCH14QAvD_BwE
개인회원 탭 클릭 완료
ID 입력 필드 발견: input[placeholder*='통합 ID']
비밀번호 입력 필드 발견
로그인 정보 입력 중...
로그인 버튼 발견: //button[text()='로그인']
로그인 버튼 클릭...
로그인 후 URL: https://www.jobkorea.co.kr/User/Profile/Start
로그인 후 제목: 프로필
✅ 로그인 성공!

=== 합격자소서 페이지로 이동 ===

페이지 151/250 처리